In [1]:
# Import Dependencies
import quandl
import pandas as pd
import csv
import scipy.stats as stats
import matplotlib.pyplot as mpl
import matplotlib.font_manager as fm
import datetime as datetime
import numpy as np
#visualizing: Correlation function (monthly)

## Obtaining City Area Codes to query with Quandl library

In [2]:
#open and read areas_city which contains area codes to run queries with Quandl
city_file = open("Resources/city_codes.txt", 'r', newline = None)
city_list = city_file.readlines()


#remove header
city_list.pop(0)

# Make a list of cities contatining city, state, and area code
cities = []
for each in range(len(city_list)):
    # Split each line in text file into a list while removing \n 
    cities.append(city_list[each].strip("\n"))

cities = [item.replace("|", ",") for item in cities]


# # split the cities list into City, State, Area Code format
cities = [each.split(",") for each in cities]

cities


[['New York', 'NY', 'New York', 'Queens', '00001'],
 ['Los Angeles', 'CA', 'Los Angeles', 'Los Angeles', '00002'],
 ['Chicago', 'IL', 'Chicago', 'Cook', '00003'],
 ['Philadelphia', 'PA', 'Philadelphia', 'Philadelphia', '00004'],
 ['Phoenix', 'AZ', 'Phoenix', 'Maricopa', '00005'],
 ['Las Vegas', 'NV', 'Las Vegas', 'Clark', '00006'],
 ['San Diego', 'CA', 'San Diego', 'San Diego', '00007'],
 ['San Jose', 'CA', 'San Jose', 'Santa Clara', '00008'],
 ['Jacksonville', 'FL', 'Jacksonville', 'Duval', '00009'],
 ['San Francisco', 'CA', 'San Francisco', 'San Francisco', '00010'],
 ['Columbus', 'OH', 'Columbus', 'Franklin', '00011'],
 ['Detroit', 'MI', 'Detroit', 'Wayne', '00012'],
 ['Charlotte', 'NC', 'Charlotte', 'Mecklenburg', '00013'],
 ['Memphis', 'TN', 'Memphis', 'Shelby', '00014'],
 ['Baltimore', 'MD', 'Baltimore', 'Baltimore City', '00015'],
 ['Boston', 'MA', 'Boston', 'Suffolk', '00016'],
 ['Seattle', 'WA', 'Seattle', 'King', '00017'],
 ['Washington', 'DC', 'Washington', 'District of Colu

In [3]:
# Read ticker_list of companies that are of interest
ticker_file = "Resources/ticker_list.csv"

tickers_df = pd.read_csv(ticker_file)
tickers_df.head()

,name,city,state,ticker,quandl_code,industry_type
0,Cardinal Health Inc.,Dublin,OH,CAH,CAH,Other
1,FedEx Corporation,Memphis,TN,FDX,FDX,Other
2,Hawaiian Holdings Inc.,Honolulu,HI,HA,HA,Other
3,Las Vegas Sands Corp.,Las Vegas,NV,LVS,LVS,Other
4,Sanderson Farms Inc.,Laurel,MS,SAFM,SAFM,Other


In [4]:
# Search through dataframe contatining companies 
# in order to obtain area codes
# Store data into city_area_code list
city_area_code = []
for each in range(len(tickers_df)):
    current_city = tickers_df.city[each]
#     print(f'Current City: {current_city}')
    current_state = tickers_df.state[each]
#     print(f'Current State: {current_state}')
    
    for each in cities:
        if current_city == each[0] and current_state == each[1]:
            city_area_code.append(each[4])
city_area_code

['00900',
 '00014',
 '00036',
 '00006',
 '09782',
 '05910',
 '10447',
 '00013',
 '00067',
 '00100',
 '00648',
 '00017',
 '00562',
 '00001',
 '10463',
 '00948',
 '00032',
 '00282',
 '02290',
 '01274']

In [5]:
# Add a new column containing city area codes to tickers dataframe
tickers_df["area_code"] = city_area_code
tickers_df.head()

,name,city,state,ticker,quandl_code,industry_type,area_code
0,Cardinal Health Inc.,Dublin,OH,CAH,CAH,Other,00900
1,FedEx Corporation,Memphis,TN,FDX,FDX,Other,00014
2,Hawaiian Holdings Inc.,Honolulu,HI,HA,HA,Other,00036
3,Las Vegas Sands Corp.,Las Vegas,NV,LVS,LVS,Other,00006
4,Sanderson Farms Inc.,Laurel,MS,SAFM,SAFM,Other,09782


In [6]:
# export file with new area code column to csv
tickers_df.to_csv("ticker_list_areacode.csv", index=False, header=True)

# Running Loops for Stock, Housing Data / Visualization

In [7]:
quandl.ApiConfig.api_key = "Me2EN7fRUzDgxznxtEk_"

In [8]:

stock_skip = []
qndl_code = tickers_df['quandl_code']
for x in qndl_code:
    if x in ['ABC', 'AAPL', 'CVS', 'NKE']:
        stock_skip.append(x)
        pass
    else:
        data_pull = quandl.get(f"EOD/{x}", start_date="2005-01-01", end_date='2015-12-31', collapse="monthly")
        data_pull.to_csv(f"API_data/{x}_FullData.csv")
        clean_df = pd.DataFrame(data_pull['Close'])
        clean_df.to_csv(f"stock_data/{x}_stock.csv")
        print(f"{x} stock data saved")
print('Data Finished')

CAH stock data saved
FDX stock data saved
HA stock data saved
LVS stock data saved
SAFM stock data saved
ANTM stock data saved
BAC stock data saved
HUM stock data saved
PFG stock data saved
AMZN stock data saved
LMT stock data saved
VZ stock data saved
XOM stock data saved
HD stock data saved
WBA stock data saved
WMT stock data saved
Data Finished


In [30]:
#ZILLOW PULL
house_code = tickers_df['area_code']
# remove leading zeroes for each area code number
house_code = [each.lstrip("0") for each in house_code]

stock_name = tickers_df['ticker']
y=0
for x in house_code:
    # Skipping housing data for ABC, CVS, and NKE
    if x in ['5910', '948', '282']:
        pass
    else:
        try:
            data_pull = quandl.get(f'ZILLOW/C{x}_MSPAH', start_date="2005-01-01", end_date="2015-12-31")
            data_pull.to_csv(f'house_data/{stock_name[y]}_housing.csv')
            print(f'{stock_name[y]} housing data pulled')
    
        except:
            print(f'{x} not found, skipping')
            pass
    y = y + 1
print('Data Finished')

CAH housing data pulled
FDX housing data pulled
HA housing data pulled
LVS housing data pulled
9782 not found, skipping
10447 not found, skipping
BAC housing data pulled
HUM housing data pulled
PFG housing data pulled
AAPL housing data pulled
AMZN housing data pulled
LMT housing data pulled
VZ housing data pulled
XOM housing data pulled
HD housing data pulled
WBA housing data pulled
WMT housing data pulled
Data Finished


In [48]:
#looping our Stock graphs
stock_fullname = tickers_df['name']
city_name = tickers_df['city']

y=0
for x in stock_name:
    try:
        stock_csv = pd.read_csv(f'stock_data/{x}_stock.csv')
        housing_csv = pd.read_csv(f'house_data/{x}_housing.csv')
        housing_price1000 = [(each/1000) for each in housing_csv['Value']]
        
    # Add housing plot
        mpl.subplot(2, 1, 1)
        mpl.plot(housing_csv['Date'], housing_price1000, color='orange')
        
        # Label graph title and y label
        mpl.title(f'{stock_fullname[y]} stock and\n{city_name[y]} housing market (2005 to 2015)', fontsize=28)
        mpl.ylabel("Median Housing Prices\nin Dollars (x1000)", labelpad=12, fontsize=20)
        
        # Set tick numbers
        mpl.xticks([]) #Remove xlabels for upper graphs
        mpl.yticks(np.arange(0, max(housing_price1000)+50, step=50), fontsize=20)
        
        #Set axes limits
        mpl.xlim(-1, len(stock_csv['Date']))
        mpl.ylim(min(housing_price1000)-50, max(housing_price1000) + 50)

    # Add Stock plot
        mpl.subplot(2, 1, 2)
        mpl.plot(stock_csv['Date'], stock_csv['Close'])

        # Label axes
        mpl.xlabel("Dates", fontsize=20)
        mpl.ylabel("Stock Close\nPrice($)", labelpad=12, fontsize=20)

        # Set axes limits
        mpl.xlim(-1, len(stock_csv['Date'])+3)
        mpl.ylim(0, max(stock_csv['Close'])+30)
        
        # Set tick numbers
        mpl.xticks(np.arange(0, len(stock_csv['Date'])+3, step=6), rotation=45, fontsize=15)
        if max(stock_csv['Close']) > 300:
            mpl.yticks(np.arange(0, max(stock_csv['Close'])+30, step=100), fontsize=20)
        else:
            mpl.yticks(np.arange(0, max(stock_csv['Close'])+30, step=30), fontsize=20)

        # set spacing between two blots
        mpl.subplots_adjust(hspace=0.1)

        # Enlarge figure size
        fig=mpl.gcf()
        fig.set_size_inches(14, 12)
        
        mpl.savefig(f'stock_graph/{x}_stock_line.png')
        mpl.clf()
    except:
        pass
                   
    y+=1

C:\Users\ricem\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


<Figure size 1008x864 with 0 Axes>

In [49]:
# Graph APPL Split data stock and housing
stock_fullname = 'Apple, Inc'
city = 'Cupertino'

stock_csv = pd.read_csv(f'AAPL_split_data.csv')
housing_csv = pd.read_csv(f'house_data/AAPL_housing.csv')
housing_price1000 = [(each/1000) for each in housing_csv['Value']]
        
# Add housing plot
mpl.subplot(2, 1, 1)
mpl.plot(housing_csv['Date'], housing_price1000, color='orange')
        
# Label graph title and y label
mpl.title(f'{stock_fullname} stock and\n{city} housing market (2005 to 2015)', fontsize=28)
mpl.ylabel("Median Housing Prices\nin Dollars (x1000)", labelpad=12, fontsize=20)

# Set tick numbers
mpl.xticks([]) #Remove xlabels for upper graphs
mpl.yticks(np.arange(0, max(housing_price1000)+50, step=50), fontsize=20)

#Set axes limits
mpl.xlim(-1, len(stock_csv['Date']))
mpl.ylim(min(housing_price1000)-50, max(housing_price1000) + 50)

# _______________________________
# Add Stock plot
mpl.subplot(2, 1, 2)
mpl.plot(stock_csv['Date'], stock_csv['Final Split'])

# Label axes
mpl.xlabel("Dates", fontsize=20)
mpl.ylabel("Stock Close\nPrice($)", labelpad=12, fontsize=20)

# Set axes limits
mpl.xlim(-1, len(stock_csv['Date'])+3)
mpl.ylim(0, max(stock_csv['Final Split'])+30)

# Set tick numbers
mpl.xticks(np.arange(0, len(stock_csv['Date'])+3, step=6), rotation=45, fontsize=15)
if max(stock_csv['Final Split']) > 300:
    mpl.yticks(np.arange(0, max(stock_csv['Final Split'])+30, step=100), fontsize=20)
else:
    mpl.yticks(np.arange(0, max(stock_csv['Final Split'])+30, step=30), fontsize=20)

# set spacing between two blots
mpl.subplots_adjust(hspace=0.1)

# Enlarge figure size
fig=mpl.gcf()
fig.set_size_inches(14, 12)

mpl.savefig(f'stock_graph/AAPL_stock_line.png')
mpl.clf()

<Figure size 1008x864 with 0 Axes>

In [12]:
import scipy.stats as stats
import numpy as np
stats.ttest_ind(clean_df['Split'], housing['Value'], equal_var=False)

KeyError: 'Split'